In [10]:
# Install all the necessary python libraries if anaconda is not able to find them.

In [11]:
# Import all the necessary python libraries
import re
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

In [12]:
# Define CONSTANTS
BASE_URL_WITH_CARI_PARAMETER = 'https://www.halal.gov.my/v4/index.php?data=ZGlyZWN0b3J5L2luZGV4X2RpcmVjdG9yeTs7Ozs=&cari='

In [13]:
# Initialize Variables
file_path = 'csv_datanb_list_part_2_cleaned.csv'
df_test = pd.read_csv(file_path, encoding='utf-8')
df_test.head(5)

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url
0,Uncle Don's (USJ Taipan),Uncle Don's,Halal Food,Subang Jaya,NaN,"55a, Jalan USJ 10/1f, Taipan Business Centre, ...",016-900 1074,https://www.google.com/maps/place/Uncle+Don%27...,NaN,http://www.uncledons.com.my/,https://www.foodpanda.my/restaurant/q3qh/uncle...
1,Malaysian Kitchen,Malaysian Kitchen,Halal Food,Subang Jaya,NaN,"14, Jalan SS 12/1b, Ss 12, 47500 Subang Jaya, ...",03-5887 2805,https://www.google.com/maps/place/Malaysian+Ki...,NaN,https://malaysiankitchen.grab.shop/,https://www.foodpanda.my/restaurant/y2xq/malay...
2,E-ONE Chinese Muslim Food Court,E-ONE Chinese Muslim Food Court,Halal Food,Subang Jaya,NaN,"12, Jalan SS 15/8a, Ss 15, 47500 Subang Jaya, ...",016-614 0060,https://www.google.com/maps/place/E-ONE+Chines...,NaN,NaN,NaN
3,Suhaimi Chinese Food (Halal),Suhaimi Chinese Food,Halal Food,Subang Jaya,Abang SS19 Food Court,"Ss 19, 40150 Subang Jaya, Selangor",NaN,https://www.google.com/maps/place/Suhaimi+Chin...,NaN,NaN,https://www.foodpanda.my/restaurant/zzu1/suhai...
4,Mohd Chan Dim Sum @ Sunway Geo Avenue,Mohd Chan Dim Sum,Halal Food,Subang Jaya,Sunway GEO Avenue,"A-01-01, Sunway Geo Avenue, Jalan Lagoon Selat...",03-5612 9939,https://www.google.com/maps/place/Mohd+Chan+Di...,NaN,http://www.mohdchan.com.my/,https://www.foodpanda.my/restaurant/xnl2/mohd-...


In [14]:
#test_full_name = df_test['Name'].iloc[226].replace(' ', '%20')
#test_simplified_name = df_test['Simplified Name'].iloc[226].replace(' ', '%20')
#print(test_simplified_name, test_full_name)

In [15]:
# url =  BASE_URL_WITH_CARI_PARAMETER + test_simplified_name

# response = requests.get(url)

# soup = BeautifulSoup(response.text, 'html.parser')

# print(soup.prettify())

In [16]:
def check_total_record(soup):
    # Find the <b> tag that contains the total record count
    total_record_tag = soup.find('b', string=lambda text: 'Total Record :' in text)

    total_records = 0
    #number_of_pages = 1
    if total_record_tag:
        try:
            # Extract the number after 'Total Record :'
            total_records = int(total_record_tag.text.split('Total Record :')[1].split('-')[0].strip())
            #number_of_pages = int(total_record_tag.text.split('From')[1].strip())         
        except ValueError:
            total_records = 0
            #print("Failed to extract the total record count.")
        finally:
            return total_records

In [32]:
def extract_company_details(soup):
    company_name = ''
    company_address = ''
    try:
        company_name_element = soup.find('span', {'class': 'company-name'})
        if company_name_element is not None:
            company_name = company_address_element.text.strip()
        else:
            company_name = None
        company_address_element = soup.find('span', {'class': 'company-address'})
        if company_address_element is not None:
            company_address = '\n'.join(segment for segment in company_address_element.stripped_strings)
            company_address = company_address.replace('\xa0', ' ').strip()
        else:
            company_address = None
    except:
        company_name = None
        company_address = None
    return company_name, company_address

In [36]:
for index, row in tqdm(df_test.iloc[1328:].iterrows(), total=df_test.iloc[1328:].shape[0], desc="Processing rows"):
    #print(f'{index+1}.')
    chinese_character_pattern = r'[\u4e00-\u9fff]+'
    total_records = 0
    full_name = row['Name']
    full_name = re.sub(chinese_character_pattern, '', full_name)
    full_name = quote(full_name)
    simplified_name = ''
    #print(len(full_name))
    #print(full_name)
    if len(full_name) != 0:
        
        simplified_name = row['Simplified Name']
        simplified_name = re.sub(chinese_character_pattern, '', simplified_name)
        simplified_name = quote(simplified_name)
        #print(simplified_name)
        url =  BASE_URL_WITH_CARI_PARAMETER + full_name
        response = requests.get(url)
        full_name_soup = BeautifulSoup(response.text, 'html.parser')
        total_records = check_total_record(full_name_soup)

    if total_records != 0 and len(full_name) != 0:

        #print('found using full name')
        company_name, company_address = extract_company_details(full_name_soup)
        df_test.at[index, 'Company Name'] = company_name
        df_test.at[index, 'Company Address'] = company_address
        #print(company_name,company_address)

    elif full_name != simplified_name and len(simplified_name) != 0:

        url =  BASE_URL_WITH_CARI_PARAMETER + simplified_name
        response = requests.get(url)
        simplified_name_soup = BeautifulSoup(response.text, 'html.parser')
        total_records = check_total_record(simplified_name_soup)

        if total_records != 0:
            #print('found using simplified name')
            company_name, company_address = extract_company_details(simplified_name_soup)
            df_test.at[index, 'Company Name'] = company_name
            df_test.at[index, 'Company Address'] = company_address
            #print(f'{company_name} \n {company_address}')

        else:
            df_test.at[index, 'Company Name'] = None
            df_test.at[index, 'Company Address'] = None
            #print('No results found.')
    else:
        df_test.at[index, 'Company Name'] = None
        df_test.at[index, 'Company Address'] = None
        #print('No results found.')
    if (index + 1) % 10 == 0:
        df_test.to_csv('csv_datanb_list_part_5_halal_company.csv', index=False)

Processing rows:   9%|▉         | 229/2488 [3:57:31<39:03:04, 62.23s/it] 


UnboundLocalError: local variable 'company_address_element' referenced before assignment

In [27]:
df_test.iloc[1359:]

,Name,Simplified Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url,Company Name,Company Address
1359,Chong Tian Restaurant | Restoran Chong Tian,Chong Tian Restaurant,Chinese Food,Subang Jaya,Hai-O Putra Heights 10,"52, Jalan Putra Mahkota 7/4b, Putra Heights, 4...",016-288 6868,https://www.google.com/maps/place/Chong+Tian+R...,NaN,NaN,NaN,NaN,NaN
1360,Hoong Foong Dim Sum,Hoong Foong Dim Sum,Chinese Food,Subang Jaya,NaN,"77 & 79 Pusat Perniagaan, Jalan USJ 21/10, Usj...",03-8081 2653,https://www.google.com/maps/place/Hoong+Foong+...,http://hoongfoong.beepit.com/,http://www.facebook.com/hoongfoongdimsum,http://hoongfoong.beepit.com/,NaN,NaN
1361,Restoran Sang Lip,Restoran Sang Lip,Chinese Food,Subang Jaya,NaN,"2, Jalan Tiara 2, Taman Perindustrian Uep, 476...",012-671 1415,https://www.google.com/maps/place/Restoran+San...,NaN,NaN,NaN,NaN,NaN
1362,Shen Heong Bak Kut Teh Restaurant,Shen Heong Bak Kut Teh Restaurant,Chinese Food,Subang Jaya,NaN,"30, Jalan SS 15/4b, Ss 15, 47500 Subang Jaya, ...",017-342 8636,https://www.google.com/maps/place/Shen+Heong+B...,NaN,https://www.facebook.com/pg/shenheongbakkutteh...,https://www.foodpanda.my/restaurant/o6dm/shen-...,NaN,NaN
1363,小龙行江湖老火锅 USJ 19 Xiao Long Xing Jiang Hu USJ 19,小龙行江湖老火锅 USJ 19 Xiao Long Xing Jiang Hu USJ 19,Chinese Food,Subang Jaya,Floor G · Rhythm Avenue,"Lot No.G-23, Ground Floor, The 19 USJ City Mal...",010-975 0539,https://www.google.com/maps/place/%E5%B0%8F%E9...,NaN,https://www.facebook.com/usjxiaolongxingjiangh...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,Curry Chicken Bun Subang Jaya,Curry Chicken Bun Subang Jaya,Dessert,Subang Jaya,NaN,"Pesiaran, Persiaran Subang Permai, USJ1, 47500...",014-722 1597,https://www.google.com/maps/place/Curry+Chicke...,NaN,http://www.twinklejelly.com/,NaN,NaN,NaN
3812,BahWang Traditional Dessert 霸王传统糖水,BahWang Traditional Dessert 霸王传统糖水,Dessert,Subang Jaya,NaN,"No. 31, Tingkat, 1, Jalan USJ 10/1e, Taipan Bu...",NaN,https://www.google.com/maps/place/BahWang+Trad...,NaN,NaN,https://www.foodpanda.my/restaurant/vyk6/bahwa...,NaN,NaN
3813,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,Dessert,Subang Jaya,NaN,"Jalan Pinggiran USJ 2/1, Taman Pinggiran Usj, ...",017-905 9614,https://www.google.com/maps/place/SKIN+DESSERT...,NaN,https://skin-dessert-shah-alam.business.site/?...,NaN,NaN,NaN
3814,Leong Sweet Dessert Kajang,Leong Sweet Dessert Kajang,Dessert,Subang Jaya,NaN,"Jalan PJS 11/9, Bandar Sunway, 47500 Subang Ja...",011-3173 0013,https://www.google.com/maps/place/Leong+Sweet+...,NaN,NaN,NaN,NaN,NaN


In [ ]:
def find_first_nan_not_none(column):
    for i, value in enumerate(column):
        # Check explicitly for np.nan and ensure it's not None
        if pd.isna(value) and value is not None:
            return i
    return None

# Find the index using the custom function
index_of_first_nan = find_first_nan_not_none(df_test['Company Name'])

# Print the result
if index_of_first_nan is not None:
    print(f"The first row with NaN (but not None) in 'company name' column is at index: {index_of_first_nan}")
    print("The row details:")
    print(df_test.iloc[index_of_first_nan])
else:
    print("No NaN (excluding None) values found in 'company name' column.")

NameError: name 'df_test' is not defined

In [ ]:
def find_first_nan_not_none(column):
    for i, value in enumerate(column):
        # Check explicitly for np.nan and ensure it's not None
        if pd.isna(value) and value is not None:
            return i
    return None

# Find the index using the custom function
index_of_first_nan = find_first_nan_not_none(df_test['Company Name'])

# Print the result
if index_of_first_nan is not None:
    print(f"The first row with NaN (but not None) in 'company name' column is at index: {index_of_first_nan}")
    print("The row details:")
    print(df_test.iloc[index_of_first_nan])
else:
    print("No NaN (excluding None) values found in 'company name' column.")

In [ ]:
def find_first_none_or_nan(column):
    for i, value in enumerate(column):
        # Check for either None or np.nan (pd.isna() covers both cases)
        if pd.isna(value):
            print(i)
            return i
    return None

# Assuming df_test is your DataFrame and 'Simplified Name' is the column of interest
index_of_first_none_or_nan = find_first_none_or_nan(df_test['Simplified Name'])

# Print the result
if index_of_first_none_or_nan is not None:
    print(f"The first row with None or NaN in 'Simplified Name' column is at index: {index_of_first_none_or_nan}")
    print("The row details:")
    print(df_test.iloc[index_of_first_none_or_nan])
else:
    print("No None or NaN values found in 'Simplified Name' column.")


1328
The first row with None or NaN in 'Simplified Name' column is at index: 1328
The row details:
Name                  | 新隆记广式烧腊| Restoran Cantonese Roast Delicacies
Simplified Name                                                  NaN
Category                                                Chinese Food
Area                                                     Subang Jaya
Located In                             USJ 1 Regalia Business Centre
Address            20, Jalan USJ 1/1b, Taman Subang Mewah, 47620 ...
Phone                                                   017-626 9638
Google Map Url     https://www.google.com/maps/place/%7C+%E6%96%B...
Menu Url           https://roastedchefhouse.eatmol.com/menu-roast...
Website Url        https://roastedchefhouse.eatmol.com/menu-roast...
Order Url                                                        NaN
Company Name                                                     NaN
Company Address                                                  NaN
Name

In [ ]:
# df_test.to_csv('csv_datanb_list_part_5_halal_company.csv',index=False,encoding='utf-8')